# Introduction to Retrieval Augmented Generation with S&P 500 news

In this notebook, you will explore how to build a simple Retrieval-Augmented Generation (RAG) pipeline using financial news articles from S&P 500 companies.

We'll start by vectorizing text data, creating a vector store using FAISS, and integrating it with OpenAI's GPT models to answer questions using retrieved information.

This workflow emulates real-world systems in finance where natural language data (news, filings, analyst reports) are used to support decision-making.

# 📌 Objectives

By the end of this notebook, students will be able to:

1. **Perform Semantic Search with Metadata Filtering:**
   - Query the provided FAISS vector store to retrieve relevant financial news articles based on natural language questions.
   - Apply optional filters using metadata such as ticker or publication date to refine search results.

2. **Enrich Data with Company Metadata:**
   - Use the `yfinance` library to retrieve company-level metadata (company name, sector, industry) for tickers in the dataset.
   - Integrate this metadata to support enhanced filtering and analysis of news data.

3. **Build a Retrieval-Augmented Generation (RAG) Pipeline:**
   - Combine retrieved news snippets as context to generate answers using OpenAI’s GPT models.
   - Construct effective prompts that guide the language model to provide concise, context-aware responses.

4. **Evaluate and Analyze RAG Outputs:**
   - Review generated answers alongside the supporting news excerpts.
   - Reflect on the strengths and limitations of the simple RAG pipeline and consider potential improvements, such as adding more filters or refining retrieval strategies.

5. **Incorporate Financial Metadata into Retrieval Context:**
   - Enrich retrieved news snippets with key financial metadata including ticker, company name, sector, and industry.
   - Format prompts that combine both text excerpts and metadata to provide richer context to the language model.

6. **Generate Context-Aware Answers Using OpenAI Models:**
   - Construct and send prompts to an LLM that leverage both news content and metadata to produce concise, informed financial analysis.

7. **Compare Answers With and Without Metadata:**
   - Evaluate the impact of including financial metadata on answer quality using criteria such as clarity, detail, accuracy, and contextual relevance.
   - Summarize findings to reflect on the role of metadata in improving retrieval-augmented generation.

## Install and Import important librairies

First, we install and import the necessary libraries for:
- Text embedding generation (sentence-transformers)
- Efficient similarity search (faiss)
- Data manipulation (pandas, numpy)
- Visualization (matplotlib)

> ℹ️ FAISS uses inner product for cosine similarity by normalizing vectors.

In [1]:
is_colab = False

try:
    import google.colab
    is_colab = True
except ImportError:
    pass

In [2]:
if is_colab:
    %pip install sentence-transformers
    %pip install faiss-gpu-cu12

In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt
import faiss
from tqdm.notebook import tqdm
import yfinance as yf
from openai import OpenAI

if is_colab:
    from google.colab import userdata
else:
    from dotenv import dotenv_values

## Load news data
We load a CSV file of financial news, focusing on TITLE and SUMMARY, along with metadata like TICKER and PUBLICATION_DATE.
These will be embedded into vectors and used for semantic retrieval.

In [4]:
K = 25

In [5]:
data_path = 'data/df_news.csv' if is_colab else '../data/df_news.csv'

In [6]:
df_news = pd.read_csv(data_path)
df_news['PUBLICATION_DATE'] = pd.to_datetime(df_news['PUBLICATION_DATE']).dt.date
display(df_news)

,TICKER,TITLE,SUMMARY,PUBLICATION_DATE,PROVIDER,URL
0,MMM,2 Dow Jones Stocks with Promising Prospects an...,The Dow Jones (^DJI) is made up of 30 of the m...,2025-05-29,StockStory,https://finance.yahoo.com/news/2-dow-jones-sto...
1,MMM,3 S&P 500 Stocks Skating on Thin Ice,The S&P 500 (^GSPC) is often seen as a benchma...,2025-05-27,StockStory,https://finance.yahoo.com/news/3-p-500-stocks-...
2,MMM,3M Rises 15.8% YTD: Should You Buy the Stock N...,"MMM is making strides in the aerospace, indust...",2025-05-22,Zacks,https://finance.yahoo.com/news/3m-rises-15-8-y...
3,MMM,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,Quarterly earnings results are a good time to ...,2025-05-22,StockStory,https://finance.yahoo.com/news/q1-earnings-rou...
4,MMM,3 Cash-Producing Stocks with Questionable Fund...,While strong cash flow is a key indicator of s...,2025-05-19,StockStory,https://finance.yahoo.com/news/3-cash-producin...
...,...,...,...,...,...,...
4866,ZTS,2 Dividend Stocks to Buy With $500 and Hold Fo...,Zoetis is a leading animal health company with...,2025-05-23,Motley Fool,https://www.fool.com/investing/2025/05/23/2-di...
4867,ZTS,Zoetis (NYSE:ZTS) Declares US$0.50 Dividend Pe...,Zoetis (NYSE:ZTS) recently affirmed a dividend...,2025-05-22,Simply Wall St.,https://finance.yahoo.com/news/zoetis-nyse-zts...
4868,ZTS,Jim Cramer on Zoetis (ZTS): “It Does Seem to B...,We recently published a list of Jim Cramer Tal...,2025-05-21,Insider Monkey,https://finance.yahoo.com/news/jim-cramer-zoet...
4869,ZTS,Zoetis (ZTS) Upgraded to Buy: Here's Why,Zoetis (ZTS) might move higher on growing opti...,2025-05-21,Zacks,https://finance.yahoo.com/news/zoetis-zts-upgr...


In [7]:
df_news['EMBEDDED_TEXT'] = df_news['TITLE'] + ' : ' + df_news['SUMMARY']

In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')

## Implement FAISS vector store
We:
- Use a pre-trained sentence transformer (all-MiniLM-L6-v2) to embed documents.
- Normalize vectors to use cosine similarity.
- Create a FAISS index and implement a basic search function.

This will allow us to retrieve relevant news snippets given a natural language question.


In [9]:
# Load model and compute embeddings
text_embeddings = model.encode(df_news['EMBEDDED_TEXT'].tolist(), convert_to_numpy=True)

# Normalize embeddings to use cosine similarity (via inner product in FAISS)
text_embeddings = text_embeddings / np.linalg.norm(text_embeddings, axis=1, keepdims=True)

# Prepare metadata
documents = df_news['EMBEDDED_TEXT'].tolist()
metadata = [
    {
        'PUBLICATION_DATE': row['PUBLICATION_DATE'],
        'TICKER': row['TICKER'],
        'PROVIDER': row['PROVIDER']
    }
    for _, row in df_news.iterrows()
]

In [10]:
embedding_dim = text_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dim)  # Cosine similarity via inner product
faiss_index.add(text_embeddings)

In [11]:
class FaissVectorStore:
    def __init__(self, model, index, embeddings, documents, metadata):
        self.model = model
        self.index = index
        self.embeddings = embeddings
        self.documents = documents
        self.metadata = metadata

    def search(self, query, k=5, metadata_filter=None):
        query_embedding = self.model.encode([query])
        query_embedding = query_embedding / np.linalg.norm(query_embedding)

        if metadata_filter:
            filtered_indices = [i for i, meta in enumerate(self.metadata) if metadata_filter(meta)]
            if not filtered_indices:
                return []
            filtered_embeddings = self.embeddings[filtered_indices]
            temp_index = faiss.IndexFlatIP(filtered_embeddings.shape[1])
            temp_index.add(filtered_embeddings)
            D, I = temp_index.search(query_embedding, k)
            indices = [filtered_indices[i] for i in I[0]]
        else:
            D, I = self.index.search(query_embedding, k)
            indices = I[0]
            D = D[0]

        results = []
        for idx, sim in zip(indices, D):
            results.append((self.documents[idx], self.metadata[idx], float(sim)))
        return results

In [12]:
# Create FAISS-based store
faiss_store = FaissVectorStore(
    model=model,
    index=faiss_index,
    embeddings=text_embeddings,
    documents=documents,
    metadata=metadata
)

### Setup OpenAI Client

👉 **Instructions**:
- Import the `OpenAI` client from the `openai` Python library.
- You will need an **OpenAI API key** to use their models programmatically:
  - Go to [https://platform.openai.com/](https://platform.openai.com/) and sign up or log in.
  - Create an API key from your [API keys dashboard](https://platform.openai.com/account/api-keys).
  - ⚠️ **Keep your API key private** and **do not** share or hardcode it in public notebooks.
- Note that **usage of the OpenAI API is not free**. You will need to:
  - Add a payment method.
  - Monitor your usage to avoid unexpected charges.
  - Optionally set usage limits from your account settings.
- You can refer to the **course’s Study Resources** for a step-by-step guide on creating an OpenAI account and retrieving your API key.

Then:
- Initialize the client with `OpenAI(api_key="YOUR_KEY_HERE")`.
- Send a test request using `.responses.create()` and the `"gpt-4o-mini"` model with a simple prompt:

  ```python
  response = client.responses.create(
      model="gpt-4o-mini",
      input="Write a one-sentence bedtime story about a unicorn."
  )
  print(response.output_text)


In [13]:
openai_key = None

if is_colab:
    openai_key = userdata.get('OPEN_AI_KEY')

else:
    config = dotenv_values('../.env')
    openai_key = config.get('OPEN_AI_KEY')

In [14]:
client = OpenAI(api_key=openai_key)

In [15]:
response = client.responses.create(
    model="gpt-4o-mini",
    input="Write a one-sentence bedtime story about a unicorn."
)
print(response.output_text)

As the stars twinkled like diamonds, a gentle unicorn named Luna danced gracefully across the moonlit meadow, spreading dreams of adventure to all the sleeping creatures nearby.


## Retrieve Additional Metadata from Yahoo Finance

👉 **Instructions**:
- We will enrich our news dataset by retrieving **company-level metadata** using the `yfinance` library.
- The goal is to map each unique stock ticker (`TICKER`) in the dataset to:
  - `COMPANY_NAME`
  - `SECTOR`
  - `INDUSTRY`

> ℹ️ `yfinance` fetches live data from Yahoo Finance. If you're running this in a cloud environment or during peak hours, expect some tickers to fail or rate limits to apply.

✅ After this step, you will have a new DataFrame (e.g. `df_meta`) with the columns `TICKER`, `COMPANY_NAME`, `SECTOR`, `INDUSTRY` that maps tickers to their company names, sectors, and industries. This metadata will be useful later to add filters and analysis based on sector or industry categories.


In [16]:
import yfinance as yf

In [17]:
ticker_meta = {
    'TICKER': [],
    'COMPANY_NAME': [],
    'SECTOR': [],
    'INDUSTRY': []
}

for ticker in tqdm(df_news['TICKER'].unique()):
    yf_ticker = yf.Ticker(ticker)
    ticker_meta['TICKER'].append(ticker)
    ticker_meta['COMPANY_NAME'].append(yf_ticker.info['longName'])
    ticker_meta['SECTOR'].append(yf_ticker.info['sector'])
    ticker_meta['INDUSTRY'].append(yf_ticker.info['industry'])

  0%|          | 0/490 [00:00<?, ?it/s]

In [18]:
df_meta = pd.DataFrame(ticker_meta)
display(df_meta)

,TICKER,COMPANY_NAME,SECTOR,INDUSTRY
0,MMM,3M Company,Industrials,Conglomerates
1,AOS,A. O. Smith Corporation,Industrials,Specialty Industrial Machinery
2,ABT,Abbott Laboratories,Healthcare,Medical Devices
3,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers - General
4,ACN,Accenture plc,Technology,Information Technology Services
...,...,...,...,...
485,XEL,Xcel Energy Inc.,Utilities,Utilities - Regulated Electric
486,XYL,Xylem Inc.,Industrials,Specialty Industrial Machinery
487,YUM,"Yum! Brands, Inc.",Consumer Cyclical,Restaurants
488,ZBH,"Zimmer Biomet Holdings, Inc.",Healthcare,Medical Devices


## Retrieval-Augmented Generation (RAG): Retrieve Documents and Generate Answers

👉 **Instructions**:

In this part of the assignment, your task is to build a simple Retrieval-Augmented Generation (RAG) pipeline that:

- Takes a user question as input.
- Searches the FAISS vector store to find a set of relevant financial news articles based on semantic similarity.
- Uses the retrieved news articles as context to generate a clear, concise answer to the question by interacting with the OpenAI language model.
- Returns both the generated answer and the underlying news snippets used for context.

### What you need to focus on:

- Implement a retrieval mechanism to query your vector store and obtain the top relevant documents for any question.
- Construct prompts that effectively combine retrieved news content with the user’s question to guide the language model’s response.
- Use the OpenAI API to generate answers grounded in the retrieved context.
- Organize the outputs so that for each question, you have:
  - The generated answer.
  - The collection of news excerpts used to produce that answer.

### What you will be provided:

- Helper functions to display outputs in markdown format.
- Lists of example questions covering topics, companies, and industries to test your implementation.

---

Your solution can take any form or structure you find appropriate, as long as it fulfills these core objectives. This exercise will give you hands-on experience with integrating retrieval and generation for practical applications in finance.


#### Print markdown
You can use the following function to print answers from GPT4o-mini in markdown.

In [19]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

#### Predefined questions

In [20]:
questions_topic = [
"What are the major concerns expressed in financial news about inflation?",
"How is investor sentiment described in recent financial headlines?",
"What role is artificial intelligence playing in recent finance-related news stories?"
]

questions_company = [
"How is Microsoft being portrayed in news stories about artificial intelligence?",
"What financial news headlines connect Amazon with automation or logistics?"
]

questions_industry = [
"What are the main themes emerging in financial news about the semiconductor industry?",
"What trends are being reported in the retail industry?",
"What risks or challenges are discussed in recent news about the energy industry?"
]

In [21]:
class RAGPipeline:
    def __init__(self, faiss_store, client):
        self.faiss_store = faiss_store
        self.client = client

    def query(self, question, k=5, metadata=None):
        relevant_documents = self.faiss_store.search(question, k=k,
                                                     metadata_filter=metadata)

        context = self._generate_prompt_context(relevant_documents)
        prompt = self._generate_prompt(question, context)
        response = self.client.responses.create(
            model="gpt-4o-mini",
            input=prompt
        )

        return response.output_text, [doc for doc, _, _ in relevant_documents]

    @staticmethod
    def _generate_prompt(question, context):
        return f"""
        You are an assistant specialized in answering finance questions.
        You will be provided with a question and additional context related
        to it, so you can have more information to generate your answers from.

        Question: {question}
        ----------------------------------------------------------------
        Context: {context}
        """

    @staticmethod
    def _generate_prompt_context(documents):
        context = "The following are recent news articles related with the question:\n\n"

        for doc, _, _ in documents:
            context += f"- {doc}\n"

        return context

In [22]:
rag = RAGPipeline(faiss_store, client)

In [23]:
def display_rag_result(question, answer, docs):
    result = f"## Q: {question}\n{answer}\n\n### Sources used for generating this answer:\n"

    docs_md = [f"- {doc}" for doc in docs]
    result += "\n".join(docs_md)

    print_markdown(result)

In [24]:
print_markdown("# Topic-focused questions")

for question in questions_topic:
    answer, docs = rag.query(question)
    display_rag_result(question, answer, docs)

# Topic-focused questions

## Q: What are the major concerns expressed in financial news about inflation?
Recent financial news has highlighted several major concerns regarding inflation:

1. **Persistent Inflation**: Reports indicate significant worry over ongoing inflationary pressures in the U.S. economy, suggesting that inflation may be more entrenched than previously anticipated. This has raised alarms about potential economic slowdowns.

2. **Food Inflation**: There is a specific focus on food inflation, which is affecting consumer prices and expectations. The rise in food costs is dampening hopes for interest rate cuts, indicating that inflation may persist longer than desired by policymakers.

3. **Impact on Monetary Policy**: Concerns about inflation are influencing the Federal Reserve's monetary policy decisions. There is trepidation that these inflation risks could hinder economic recovery, leading to a cautious approach regarding interest rate adjustments.

4. **Tariff Influences**: Ongoing tariff disputes and changes are complicating the inflation landscape by affecting supply chains and pricing, further contributing to inflationary pressures on consumer goods.

Overall, the financial news underscores a multifaceted view of inflation's impact on the economy, suggesting an intricate balancing act for policymakers.

### Sources used for generating this answer:
- Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown.
- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead
- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead
- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead
- The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead

## Q: How is investor sentiment described in recent financial headlines?
Investor sentiment in recent financial headlines appears to be a mix of cautious optimism and underlying skepticism. Analysts are frequently highlighting stocks that show potential for significant upside, driven by factors like strong fundamentals, positive news, or market interest. However, there is also a consistent warning about the tendency of analysts to be overly bullish, often for institutional or business-related reasons. 

For example:

- Articles mention ambitious price targets, suggesting confidence in certain stocks, but advise investors to remain skeptical due to potential institutional pressures that could skew forecasts.
- Some stocks are noted for their market outperformance, attributed to positive developments and retail enthusiasm (e.g., from platforms like Reddit), yet there's a reminder to evaluate these stocks critically.
- There are significant cautions when analysts become bearish or express concerns about stocks, as these calls are less common and indicate a serious reconsideration of the stock’s potential.
- The overarching theme suggests investor sentiment is characterized by excitement around potential growth but tempered by an awareness of possible biases in analyst ratings. 

Overall, while there is evident enthusiasm in the market, both from analysts and retail investors, the narrative encourages a balanced approach, urging investors to critically assess the underlying fundamentals and motivations behind the analyses.

### Sources used for generating this answer:
- 3 of Wall Street’s Favorite Stocks Facing Headwinds : Wall Street has set ambitious price targets for the stocks in this article. While this suggests attractive upside potential, it’s important to remain skeptical because analysts face institutional pressures that can sometimes lead to overly optimistic forecasts.
- 3 Hyped Up  Stocks Facing Headwinds : Great things are happening to the stocks in this article. They’re all outperforming the market over the last month because of positive catalysts such as a new product line, constructive news flow, or even a loyal Reddit fanbase.
- 1 of Wall Street’s Favorite Stock with Impressive Fundamentals and 2 to Think Twice About : The stocks in this article have caught Wall Street’s attention in a big way, with price targets implying returns above 20%. But investors should take these forecasts with a grain of salt because analysts typically say nice things about companies so their firms can win business in other product lines like M&A advisory.
- 1 Unpopular Stock that Should Get More Attention and 2 to Steer Clear Of : When Wall Street turns bearish on a stock, it’s worth paying attention. These calls stand out because analysts rarely issue grim ratings on companies for fear their firms will lose out in other business lines such as M&A advisory.
- 3 of Wall Street’s Favorite Stocks with Questionable Fundamentals : Wall Street is overwhelmingly bullish on the stocks in this article, with price targets suggesting significant upside potential. However, it’s worth remembering that analysts rarely issue sell ratings, partly because their firms often seek other business from the same companies they cover.

## Q: What role is artificial intelligence playing in recent finance-related news stories?
Artificial intelligence (AI) is increasingly playing a pivotal role in the finance sector, as highlighted by recent news stories. Here are some notable developments:

1. **Enhanced Lending Solutions**: Companies like Jack Henry & Associates are integrating AI-driven lending technologies to improve productivity and reduce human error in financial services. This is indicative of a broader trend where traditional financial institutions are leveraging AI to streamline operations and enhance customer experiences.

2. **Investment Growth**: Major corporations, such as Meta Platforms, are investing heavily in AI applications. These investments are expected to support growth in stocks tied to AI, suggesting that investors are closely watching how these technologies may impact future profitability.

3. **Stock Market Dynamics**: Noteworthy hedge fund managers are buying stocks like Palantir and Upstart, anticipating significant returns fueled by AI technologies. Both companies are leveraging AI to address critical financial applications, such as credit risk assessment, which aligns with a growing demand for data-driven decision-making in finance.

4. **Market Resilience**: Despite volatility in the broader stock market, companies like Palantir have shown strong stock performance, benefiting from their AI capabilities. This illustrates that firms utilizing AI can potentially weather market fluctuations better than those that don't.

5. **Generative AI Applications**: The rise of generative AI is highlighted in various sectors, including content creation for companies like Netflix. While not purely financial, the success in leveraging AI for content generation foreshadows similar applications in financial analysis and reporting.

In summary, AI is transforming finance by enhancing operational efficiency, driving investments, creating market opportunities, and contributing to stock performance stability, making it a key area of focus for investors and companies alike.

### Sources used for generating this answer:
- Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […]
- This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making.
- Billionaires Are Buying 2 Artificial Intelligence (AI) Stocks That Wall Street Analysts Say Can Soar Up to 240% : Several billionaire hedge fund managers bought shares of Palantir and/or Upstart in the first quarter -- stocks where certain analysts anticipate substantial upside.  Palantir is successfully tapping demand for artificial intelligence (AI) with government and commercial customers, but the stock trades at a very expensive valuation.  Upstart is generating attractive returns for lenders by helping them quantify credit risk with artificial intelligence, and the stock trades at a very reasonable valuation.
- Better Artificial Intelligence (AI) Stock: Palantir vs. Snowflake : Shares of both Palantir and Snowflake have delivered healthy gains in 2025 despite the broader stock market weakness.  Palantir stock has shot up 63% this year despite bouts of volatility.  Palantir Technologies helps commercial and government clients integrate generative AI capabilities into their operations with its Artificial Intelligence Platform (AIP), which was launched roughly two years ago.
- 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards.

In [25]:
print_markdown("# Company-focused questions")

for question in questions_company:
    answer, docs = rag.query(question)
    display_rag_result(question, answer, docs)

# Company-focused questions

## Q: How is Microsoft being portrayed in news stories about artificial intelligence?
In recent news stories about artificial intelligence, Microsoft is being portrayed as a significant player in the tech industry, particularly regarding its investments and innovations in AI technologies. Although the context provided does not mention Microsoft directly, the overall landscape of discussions around AI reveals that companies like Microsoft are keenly focusing on AI advancements, especially given the competitive environment.

The mention of other tech giants, such as Meta and Salesforce, suggests a backdrop where Microsoft is likely being compared or aligned with these leaders in terms of embracing AI potential. Analysts and experts may see Microsoft's AI initiatives as crucial for its growth and market position, reflecting a positive outlook if the investments yield substantial results.

Overall, while specific articles in your context don't touch on Microsoft, it is reasonable to infer that the company is generally portrayed in a favorable light as part of the broader narrative of companies aggressively pursuing AI innovations for future growth.

### Sources used for generating this answer:
- This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making.
- How Salesforce has 'overcorrected' by leaning into AI : D.A. Davidson head of technology research Gil Luria joins Market Domination to discuss Salesforce (CRM) earnings and the company's trajectory. Luria says Salesforce is "too focused" on artificial intelligence (AI), as the other parts of its business "rapidly" decelerate and the company loses market share to competitors. Luria has the equivalent of a Sell rating on the stock. To watch more expert insights and analysis on the latest market action, check out more Market Domination here.
- Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […]
- 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards.
- Meta (META) AI Reaches 1 Billion Users, Eyes Paid Features and Subscriptions : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Meta Platforms, Inc. (NASDAQ:META) stands against other AI stocks on Wall Street’s radar. Meta Platforms, Inc. (NASDAQ:META) is a global technology company. On May 28, CNBC reported that Meta Platforms, Inc. (NASDAQ:META)’s artificial […]

## Q: What financial news headlines connect Amazon with automation or logistics?
Here are some financial news headlines connecting Amazon with automation or logistics:

1. **Amazon's AI Roadmap With AWS CEO Garman**: This article highlights how Amazon is leveraging artificial intelligence across its operations, which includes logistics and automation, to enhance efficiency and operations.

2. **Truist Reiterates Buy on Amazon.com as Q2 Revenue Tracks Ahead**: This suggests that Amazon's initiatives in automation and AI could contribute to revenue growth, indicating a positive outlook tied to their logistics capabilities.

These articles underline Amazon's ongoing integration of AI and automation technologies to streamline its logistics and improve overall business performance.

### Sources used for generating this answer:
- Truist Reiterates Buy on Amazon.com (AMZN) as Q2 Revenue Tracks Ahead : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Amazon.com Inc. (NASDAQ:AMZN) stands against other AI stocks on Wall Street’s radar. Amazon.com Inc. (NASDAQ:AMZN) is an American technology company offering e-commerce, cloud computing, and other services, including digital streaming […]
- Amazon's AI Roadmap With AWS CEO Garman : Every aspect of Amazon is leveraging artificial intelligence, says Matt Garman, CEO of Amazon Web Services. Garman discusses Amazon's AI roadmap and reflects on his first year in the role with Ed Ludlow on "Bloomberg Technology."
- Woodward's Volumes, Automation to Drive Earnings Growth, Truist Says : Woodward's (WWD) increasing volumes, pricing, automation, and products will push its aerospace margi
- Top Stock Reports for Amazon.com, Johnson & Johnson & Cisco Systems : Today's Research Daily features new research reports on 16 major stocks, including Amazon.com, Inc. (AMZN), Johnson & Johnson (JNJ) and Cisco Systems, Inc. (CSCO), as well as a micro-cap NeurAxis, Inc. (NRXS).
- Winners And Losers Of Q1: C.H. Robinson Worldwide (NASDAQ:CHRW) Vs The Rest Of The Air Freight and Logistics Stocks : As the craze of earnings season draws to a close, here’s a look back at some of the most exciting (and some less so) results from Q1. Today, we are looking at air freight and logistics stocks, starting with C.H. Robinson Worldwide (NASDAQ:CHRW).

In [26]:
print_markdown("# Industry-focused questions")

for question in questions_industry:
    answer, docs = rag.query(question)
    display_rag_result(question, answer, docs)

# Industry-focused questions

## Q: What are the main themes emerging in financial news about the semiconductor industry?
The recent financial news surrounding the semiconductor industry highlights several emerging themes:

1. **Revenue Trends and International Markets**: There's a significant focus on international revenue trends for companies like ON Semiconductor Corp. This indicates a growing interest in understanding how global markets are impacting financial performance and the forecasts set by Wall Street.

2. **Investor Sentiment**: There appears to be a pronounced interest among investors in stocks like ON Semiconductor, as indicated by increased attention from platforms like Zacks.com. This suggests that investor sentiment is leaning toward optimism, even in the face of challenges such as soft earnings reports.

3. **Earnings Season Insights**: The end of the earnings season is being used as an opportunity to evaluate stocks in the semiconductor sector, such as Vishay Intertechnology and Analog Semiconductors. This highlights a trend where investors look for potential winners based on recent performance and market conditions.

4. **Stock Potential from High-Profile Investors**: The mention of ON Semiconductor in the context of billionaire investors like Glenn Russell Dubin indicates that high-profile investor picks are becoming a point of interest, signaling confidence in the stock's potential upsides.

5. **Market Evaluation**: Overall, there's a broad assessment of how semiconductor companies are navigating the current business environment, influencing stock evaluations and investor strategies as they look for growth opportunities.

These themes reflect a nuanced view of the semiconductor industry, characterized by both optimism and careful scrutiny of performance metrics.

### Sources used for generating this answer:
- Investing in ON Semiconductor Corp. (ON)? Don't Miss Assessing Its International Revenue Trends : Explore ON Semiconductor Corp.'s (ON) international revenue trends and how these numbers impact Wall Street's forecasts and what's ahead for the stock.
- ON Semiconductor Corporation (ON) is Attracting Investor Attention: Here is What You Should Know : Recently, Zacks.com users have been paying close attention to ON Semiconductor Corp. (ON). This makes it worthwhile to examine what the stock has in store.
- Some May Be Optimistic About ON Semiconductor's (NASDAQ:ON) Earnings : Soft earnings didn't appear to concern ON Semiconductor Corporation's ( NASDAQ:ON ) shareholders over the last week...
- Spotting Winners: Vishay Intertechnology (NYSE:VSH) And Analog Semiconductors Stocks In Q1 : The end of an earnings season can be a great time to discover new stocks and assess how companies are handling the current business environment. Let’s take a look at how Vishay Intertechnology (NYSE:VSH) and the rest of the analog semiconductors stocks fared in Q1.
- ON Semiconductor (ON): Among Billionaire Glenn Russell Dubin’s Stock Picks with Huge Upside Potential : We recently published a list of Billionaire Glenn Russell Dubin’s 10 Stock Picks with Huge Upside Potential. In this article, we are going to take a look at where ON Semiconductor Corporation (NASDAQ:ON) stands against Billionaire Glenn Russell Dubin’s other stock picks with huge upside potential. Glenn Russell Dubin is one of the industry’s most […]

## Q: What trends are being reported in the retail industry?
Recent trends in the retail industry indicate a significant adaptation to technological changes and shifting consumer behavior. Here are some key points based on the provided context:

1. **Shift in Business Models**: Retailers are increasingly modifying their business strategies to align with technological advancements that are transforming shopping experiences. This includes the integration of e-commerce and enhanced online shopping platforms.

2. **Volatility in Demand**: The retail sector remains vulnerable to fluctuations in consumer spending, leading to uncertainty in the market. Recent statistics show a notable decline in retail stocks, with a drop of approximately 12.3% to 13.7% over the past six months, reflecting a performance worse than that of the S&P 500.

3. **Impact of Trade Policies**: Changes in trade policies, particularly tariffs from previous administrations, have prompted retailers to reassess their supply chains. Executives are actively managing cost impacts related to legal rulings and adjusting pricing strategies accordingly.

4. **Economic Cycle Reliance**: Many consumer discretionary businesses in retail are closely tied to economic cycles. With recent economic indicators showing weakness, these companies are struggling to maintain customer demand, further contributing to their stock performance decline.

Overall, the retail industry is navigating a landscape marked by technological adaptation, economic challenges, and evolving supply chain strategies amidst fluctuating consumer demand.

### Sources used for generating this answer:
- 3 Consumer Stocks That Concern Us : Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of consumer spending. This has stirred some uncertainty lately as retail stocks have tumbled by 13.7% over the past six months. This performance was worse than the S&P 500’s 5.5% loss.
- Retailers, Ducking Trade-War Curveballs, Stick to Their Plans : As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increases already have hit shelves.
- 3 Consumer Stocks Skating on Thin Ice : The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand trends are working against their favor as the industry has tumbled by 12.3%. This drop was significantly worse than the S&P 500’s 2.1% decline.
- Packaging Corporation of America (NYSE:PKG) Hasn't Managed To Accelerate Its Returns : What trends should we look for it we want to identify stocks that can multiply in value over the long term? Amongst...
- Air Products and Chemicals (NYSE:APD) Will Be Hoping To Turn Its Returns On Capital Around : To find a multi-bagger stock, what are the underlying trends we should look for in a business? Firstly, we'll want to...

## Q: What risks or challenges are discussed in recent news about the energy industry?
Recent news indicates several significant risks and challenges facing the energy industry:

1. **Legislative Risks for Renewable Energy**: The U.S. Congress is advancing a bill that could repeal key subsidies for the renewable energy sector. This poses a major threat to renewable energy projects that may become uneconomical without financial support, causing significant stock crashes among renewable energy companies.

2. **Oil Price Volatility**: Companies in the oilfield service sector, such as SLB, HAL, and BKR, are facing challenges due to sliding oil prices. A decrease in oil prices often leads to reduced investment in drilling and exploration, affecting the profitability and sustainability of these companies.

3. **Tariff Impacts**: Rising tariffs are adding to the financial strain on the energy sector, particularly in oilfield services. Increased costs from tariffs may negatively affect operational budgets and overall profitability, particularly for companies reliant on imported resources.

4. **Economic Uncertainty**: As drilling budgets shrink, there is uncertainty about future investments in both traditional and renewable energy sources. The outlook for growth in demand for liquefied natural gas (LNG) and emerging technologies such as AI may provide some support, but reliance on these factors introduces additional risk.

Overall, the energy industry is navigating a complex landscape of legislative threats, volatile commodity prices, trade tensions, and economic uncertainties, all of which may hinder its growth and stability in the near future.

### Sources used for generating this answer:
- Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.
- Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.
- Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?
- Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?
- 3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to close loopholes that will improve its competitive positioning.  It's difficult to predict precisely what the tariff landscape will look like when the dust settles on the trade conflict, but we can say some things with a high degree of certainty.

## Analysis & Questions - Section 1

### Analysis and Reflection on Retrieval and Generation Results
After running the RAG pipeline and obtaining answers along with their supporting news excerpts, take some time to carefully review both the generated responses and the retrieved contexts.

- **For each question, read the answer and then the corresponding news snippets used as context.**

- Reflect on the following points and document your observations:
1. **Relevance**
2. **Completeness**  
3. **Bias or Noise**
4. **Consistency**  
5. **Improvement Ideas**   

and answer the questions below:

#### **Question 1.** How well do the retrieved news snippets support the generated answer? Are the key facts or themes in the answer clearly grounded in the context?

YOUR WRITTEN RESPONSE HERE


#### **Question 2.** Does the answer fully address the question, or does it leave important aspects out? Consider if the retrieved context provided enough information to generate a thorough response.

YOUR WRITTEN RESPONSE HERE
  

#### **Question 3.** Are there any irrelevant or misleading snippets retrieved that may have influenced the answer? How might this affect the quality of the output?

YOUR WRITTEN RESPONSE HERE


#### **Question 4.**  Do the news snippets show consistent information, or are there conflicting viewpoints? How does the LLM handle potential contradictions in the context?

YOUR WRITTEN RESPONSE HERE
  

#### **Question 5.**  Based on your observations, suggest ways the retrieval or generation process could be improved (e.g., better filtering, adjusting `k`, refining prompt design).

YOUR WRITTEN RESPONSE HERE
  

## 🧠 Retrieval-Augmented Generation (RAG) v2: Adding Financial Metadata to Improve Generation

👉 **Instructions**:

In this part of the assignment, you’ll enhance your Retrieval-Augmented Generation (RAG) pipeline by incorporating *financial metadata* to provide more contextually rich answers.

Your goal is to evaluate whether metadata such as **company name**, **sector**, and **industry** helps the LLM generate **more accurate and grounded answers** to financial questions.

---

### ✅ What your updated pipeline should do:

- Retrieve relevant financial news articles using semantic similarity with FAISS.
- Enrich each retrieved document with financial metadata:
  - Ticker symbol
  - Full company name
  - Sector (e.g., Technology, Energy)
  - Industry (e.g., Semiconductors, Retail)
- Construct prompts that include both:
  - Retrieved news text
  - Associated metadata
- Send the prompt to the OpenAI model to generate an informed response.
- Return:
  - The final answer
  - The exact set of contextual documents used to produce that answer

---

### 🧪 Evaluation and Comparison:

You will test your improved RAG pipeline on the same three types of questions provided earlier:
- **Topic-focused** (e.g., inflation, interest rates)
- **Company-focused** (e.g., questions about Tesla, Nvidia)
- **Industry-focused** (e.g., semiconductors, utilities)


In [27]:
class EnrichedRAGPipeline:
    def __init__(self, faiss_store, client, company_metadata):
        self.faiss_store = faiss_store
        self.client = client
        self.company_metadata = company_metadata

    def query(self, question, k=5, metadata=None):
        relevant_documents = self.faiss_store.search(question, k=k,
                                                     metadata_filter=metadata)

        context = self._generate_prompt_context(relevant_documents)
        prompt = self._generate_prompt(question, context)
        response = self.client.responses.create(
            model="gpt-4o-mini",
            input=prompt
        )

        return response.output_text, [(doc, self._get_company_metadata(meta['TICKER'])) for doc, meta, _ in relevant_documents]

    @staticmethod
    def _generate_prompt(question, context):
        return f"""
        You are an assistant specialized in answering finance questions.
        You will be provided with a question and additional context related
        to it, so you can have more information to generate your answers from.
        This added context will take the form of recent news article related to
        the question. Each of these articles will be about a specific
        company, sector or industry.

        Question: {question}
        ----------------------------------------------------------------
        Context: {context}
        """

    def _generate_prompt_context(self, documents):
        context = "The following are recent news articles related with the question:\n\n"

        for doc, meta, _ in documents:
            ticker = meta['TICKER']
            c_meta = self._get_company_metadata(ticker)
            context += f"This following news article is related to the '{c_meta['COMPANY_NAME']}' company ({ticker}), operating in the {c_meta['SECTOR']} sector and the {c_meta['INDUSTRY']} industry."
            context += f"> {doc}\n"

        return context

    def _get_company_metadata(self, ticker):
        return self.company_metadata[self.company_metadata['TICKER'] == ticker].to_dict(orient='records')[0]

In [28]:
enriched_rag = EnrichedRAGPipeline(faiss_store, client, df_meta)

In [29]:
def display_enriched_rag_result(question, answer, docs):
    result = f"## Q: {question}\n{answer}\n\n### The following news articles were used for generating this answer:\n"
    result += "| Ticker | Company | Sector | Industry | Article |\n"
    result += "|--------|--------------|--------|----------|-----|\n"
    for doc, meta in documents:
        result += f"| {meta['TICKER']} | {meta['COMPANY_NAME']} | {meta['SECTOR']} | {meta['INDUSTRY']} | {doc} |\n"
    print_markdown(result)

In [30]:
print_markdown("# Topic-focused questions")
for question in questions_topic:
    answer, documents = enriched_rag.query(question)
    display_enriched_rag_result(question, answer, documents)

# Topic-focused questions

## Q: What are the major concerns expressed in financial news about inflation?
Recent financial news highlights several major concerns surrounding inflation:

1. **Persistent US Inflation**: The Federal Reserve's recent minutes reveal growing anxiety over ongoing inflationary pressures, raising fears of a potential economic slowdown.

2. **Food Inflation Impact**: Rising food prices are causing significant worry among consumers and investors. This persistent food inflation dampens expectations for interest rate cuts, suggesting that the Fed may maintain or increase rates to combat inflation rather than ease them.

3. **Economic Policy Uncertainty**: Related to the concerns over inflation are the ongoing tariff discussions and their potential impacts on different sectors, creating a climate of uncertainty that could complicate economic recovery.

4. **Sector-Specific Impacts**: Different sectors, such as consumer cyclical and technology, are expressing concerns about how inflation is affecting their operations and profitability. Companies like Tesla and lululemon are finding that increased costs can hurt consumer purchasing power, which subsequently impacts sales.

Overall, the overarching theme in the financial news is that inflation remains a significant and multifaceted threat, prompting stakeholders to reassess their economic strategies and expectations.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| BLK | BlackRock, Inc. | Financial Services | Asset Management | Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown. |
| TSLA | Tesla, Inc. | Consumer Cyclical | Auto Manufacturers | The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead |
| NVDA | NVIDIA Corporation | Technology | Semiconductors | The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead |
| LULU | lululemon athletica inc. | Consumer Cyclical | Apparel Retail | The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead |
| AVGO | Broadcom Inc. | Technology | Semiconductors | The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead |


## Q: How is investor sentiment described in recent financial headlines?
Investor sentiment in recent financial headlines reflects a mixture of optimism and skepticism, primarily influenced by analyst ratings and market performance.

1. **Skeptical Optimism**: There are instances where analysts set ambitious price targets for stocks, suggesting potential upside. However, the accompanying commentary emphasizes the necessity of caution, particularly due to institutional pressures that may lead to overly optimistic forecasts.

2. **Positive Catalysts**: Some stocks have shown strong performance due to positive developments such as new product launches, favorable news, or a dedicated following among retail investors. This indicates a bullish sentiment about specific companies, especially in sectors like technology.

3. **Grim Predictions**: On the flip side, bearish sentiments are highlighted when analysts caution against certain stocks, calling attention to predictions that diverge from the more common positive outlook. These warnings are noteworthy as they're issued less frequently, indicating potential serious concerns about those stocks.

4. **Cautious Excitement**: Despite the overall bullishness on certain stocks, analysts frequently warn investors to remain vigilant, reminding them that companies with impressive fundamentals may not always align with market expectations due to analysts' tendencies to maintain positive views for business reasons.

Overall, while there are signs of excitement in the market, especially in sectors experiencing growth, a general caution is advised, reflecting a more complex view of investor sentiment.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| KMX | CarMax, Inc. | Consumer Cyclical | Auto & Truck Dealerships | 3 of Wall Street’s Favorite Stocks Facing Headwinds : Wall Street has set ambitious price targets for the stocks in this article. While this suggests attractive upside potential, it’s important to remain skeptical because analysts face institutional pressures that can sometimes lead to overly optimistic forecasts. |
| MCHP | Microchip Technology Incorporated | Technology | Semiconductors | 3 Hyped Up  Stocks Facing Headwinds : Great things are happening to the stocks in this article. They’re all outperforming the market over the last month because of positive catalysts such as a new product line, constructive news flow, or even a loyal Reddit fanbase. |
| MPWR | Monolithic Power Systems, Inc. | Technology | Semiconductors | 1 of Wall Street’s Favorite Stock with Impressive Fundamentals and 2 to Think Twice About : The stocks in this article have caught Wall Street’s attention in a big way, with price targets implying returns above 20%. But investors should take these forecasts with a grain of salt because analysts typically say nice things about companies so their firms can win business in other product lines like M&A advisory. |
| DRI | Darden Restaurants, Inc. | Consumer Cyclical | Restaurants | 1 Unpopular Stock that Should Get More Attention and 2 to Steer Clear Of : When Wall Street turns bearish on a stock, it’s worth paying attention. These calls stand out because analysts rarely issue grim ratings on companies for fear their firms will lose out in other business lines such as M&A advisory. |
| RVTY | Revvity, Inc. | Healthcare | Diagnostics & Research | 3 of Wall Street’s Favorite Stocks with Questionable Fundamentals : Wall Street is overwhelmingly bullish on the stocks in this article, with price targets suggesting significant upside potential. However, it’s worth remembering that analysts rarely issue sell ratings, partly because their firms often seek other business from the same companies they cover. |


## Q: What role is artificial intelligence playing in recent finance-related news stories?
Artificial intelligence (AI) is increasingly shaping the finance and tech industries, as highlighted in several recent news stories. Here's a summary of the roles AI is playing in these contexts:

1. **Enhanced Lending Technology**: Jack Henry & Associates, a technology firm, is integrating AI-driven lending technologies with its systems, aiming to improve productivity and reduce human errors in financial transactions. This reflects a broader trend where AI tools are being adopted to streamline processes in financial services.

2. **Investment Valuations with AI**: Meta Platforms is leveraging AI investments to potentially transform its business model beyond traditional internet content. Investors are closely watching how these AI applications can enhance company valuations, indicating a market expectation for AI’s impact on revenue generation and performance.

3. **Risk Assessment and Credit Scoring**: Palantir Technologies is applying AI to help clients, including government entities and commercial businesses, assess credit risk effectively, particularly through its platform aimed at quantifying risk. This illustrates how financial institutions can utilize AI for better decision-making in lending.

4. **Generative AI in Content Production**: Netflix is exploring generative AI to expedite content creation, realizing that faster and more efficient content production could provide a competitive edge in attracting and retaining subscribers. This shows how AI can influence revenues in the entertainment sector, which has implications for finance through improved cash flows.

5. **Stock Performance and Market Speculation**: Both Palantir and Upstart are experiencing strong analyst interest and stock price movements due in part to their innovations in AI. The speculative nature among investors indicates that successful AI integration could lead to significant stock appreciation.

Overall, AI is driving transformational changes in productivity, operational efficiency, and market valuations across various finance-related sectors, impacting how companies strategize and position themselves in competitive landscapes.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| JKHY | Jack Henry & Associates, Inc. | Technology | Information Technology Services | Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […] |
| META | Meta Platforms, Inc. | Communication Services | Internet Content & Information | This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making. |
| PLTR | Palantir Technologies Inc. | Technology | Software - Infrastructure | Billionaires Are Buying 2 Artificial Intelligence (AI) Stocks That Wall Street Analysts Say Can Soar Up to 240% : Several billionaire hedge fund managers bought shares of Palantir and/or Upstart in the first quarter -- stocks where certain analysts anticipate substantial upside.  Palantir is successfully tapping demand for artificial intelligence (AI) with government and commercial customers, but the stock trades at a very expensive valuation.  Upstart is generating attractive returns for lenders by helping them quantify credit risk with artificial intelligence, and the stock trades at a very reasonable valuation. |
| PLTR | Palantir Technologies Inc. | Technology | Software - Infrastructure | Better Artificial Intelligence (AI) Stock: Palantir vs. Snowflake : Shares of both Palantir and Snowflake have delivered healthy gains in 2025 despite the broader stock market weakness.  Palantir stock has shot up 63% this year despite bouts of volatility.  Palantir Technologies helps commercial and government clients integrate generative AI capabilities into their operations with its Artificial Intelligence Platform (AIP), which was launched roughly two years ago. |
| NFLX | Netflix, Inc. | Communication Services | Entertainment | 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards. |


In [31]:
print_markdown("# Company-focused questions")
for question in questions_company:
    answer, documents = enriched_rag.query(question)
    display_enriched_rag_result(question, answer, documents)

# Company-focused questions

## Q: How is Microsoft being portrayed in news stories about artificial intelligence?
In the recent news articles provided, Microsoft is not specifically mentioned; however, there are a few important themes regarding AI investments and market perceptions of tech companies that may reflect the overall environment in which Microsoft operates, particularly in relation to artificial intelligence.

1. **Comparisons with Rival Companies**: Other tech companies, such as Meta and Salesforce, are actively investing in AI technologies. Meta, for instance, is being recognized for its substantial AI initiatives, having reached a significant milestone of 1 billion users for its AI features. This context suggests that Microsoft, as a key player in the tech sector, is likely being framed similarly when it comes to its AI initiatives.

2. **Market Reactions**: Articles highlight varying sentiments towards AI strategies within tech firms. For instance, Salesforce has faced criticism for being overly focused on AI to the detriment of its core business. This raises the question of how Microsoft’s AI investments are perceived in terms of balance and strategic execution.

3. **Impact on Valuation**: Investment discussions surrounding AI indicate that companies are being valued based on their AI prospects. While Microsoft is not directly referenced, the sentiment towards companies exploring AI like Meta could similarly influence how analysts and investors regard Microsoft's stock, especially if its AI initiatives are perceived as impactful.

Overall, while the specific portrayal of Microsoft in AI stories is not detailed in the articles provided, it can be inferred that Microsoft is likely part of the broader conversation around AI growth and investment strategy among tech giants. This environment is characterized by enthusiasm for AI applications but also caution regarding overcommitment that might affect broader business performance.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| META | Meta Platforms, Inc. | Communication Services | Internet Content & Information | This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making. |
| CRM | Salesforce, Inc. | Technology | Software - Application | How Salesforce has 'overcorrected' by leaning into AI : D.A. Davidson head of technology research Gil Luria joins Market Domination to discuss Salesforce (CRM) earnings and the company's trajectory. Luria says Salesforce is "too focused" on artificial intelligence (AI), as the other parts of its business "rapidly" decelerate and the company loses market share to competitors. Luria has the equivalent of a Sell rating on the stock. To watch more expert insights and analysis on the latest market action, check out more Market Domination here. |
| JKHY | Jack Henry & Associates, Inc. | Technology | Information Technology Services | Jack Henry (JKHY) Integrates AI-Driven Lending Tech With Algebrik : We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we are going to take a look at where Jack Henry & Associates, Inc. (NASDAQ:JKHY) stands against other AI news Investors should not miss this week. Artificial Intelligence (AI) is known to increase productivity, decrease human error, […] |
| NFLX | Netflix, Inc. | Communication Services | Entertainment | 2 Underrated Artificial Intelligence (AI) Stocks to Buy and Hold : Generative AI can simplify and speed up many tasks, including content production.  It's easy to see the potential for Netflix, whose content strategy is integral to its success.  Netflix's creations have attracted millions of viewers and won many awards. |
| META | Meta Platforms, Inc. | Communication Services | Internet Content & Information | Meta (META) AI Reaches 1 Billion Users, Eyes Paid Features and Subscriptions : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Meta Platforms, Inc. (NASDAQ:META) stands against other AI stocks on Wall Street’s radar. Meta Platforms, Inc. (NASDAQ:META) is a global technology company. On May 28, CNBC reported that Meta Platforms, Inc. (NASDAQ:META)’s artificial […] |


## Q: What financial news headlines connect Amazon with automation or logistics?
Recent financial news headlines connecting Amazon with automation or logistics include:

1. **Amazon's AI Roadmap with AWS CEO Garman**: In this article, it is highlighted that every aspect of Amazon is leveraging artificial intelligence, indicating a strong focus on automation in logistics and other operations through advanced technology.

2. **Truist Reiterates Buy on Amazon.com**: Although primarily about Amazon's financial performance, this piece mentions Amazon in the context of being on Wall Street's radar for AI investments, indirectly tying into how automation might impact its logistics and operational efficiencies.

These articles underline Amazon's integration of automation and AI technologies, which are critical for improving logistics and operational performance in its extensive e-commerce network.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| TFC | Truist Financial Corporation | Financial Services | Banks - Regional | Truist Reiterates Buy on Amazon.com (AMZN) as Q2 Revenue Tracks Ahead : We recently published a list of 10 AI Stocks on Wall Street’s Radar. In this article, we are going to take a look at where Amazon.com Inc. (NASDAQ:AMZN) stands against other AI stocks on Wall Street’s radar. Amazon.com Inc. (NASDAQ:AMZN) is an American technology company offering e-commerce, cloud computing, and other services, including digital streaming […] |
| AMZN | Amazon.com, Inc. | Consumer Cyclical | Internet Retail | Amazon's AI Roadmap With AWS CEO Garman : Every aspect of Amazon is leveraging artificial intelligence, says Matt Garman, CEO of Amazon Web Services. Garman discusses Amazon's AI roadmap and reflects on his first year in the role with Ed Ludlow on "Bloomberg Technology." |
| TFC | Truist Financial Corporation | Financial Services | Banks - Regional | Woodward's Volumes, Automation to Drive Earnings Growth, Truist Says : Woodward's (WWD) increasing volumes, pricing, automation, and products will push its aerospace margi |
| AON | Aon plc | Financial Services | Insurance Brokers | Top Stock Reports for Amazon.com, Johnson & Johnson & Cisco Systems : Today's Research Daily features new research reports on 16 major stocks, including Amazon.com, Inc. (AMZN), Johnson & Johnson (JNJ) and Cisco Systems, Inc. (CSCO), as well as a micro-cap NeurAxis, Inc. (NRXS). |
| CHRW | C.H. Robinson Worldwide, Inc. | Industrials | Integrated Freight & Logistics | Winners And Losers Of Q1: C.H. Robinson Worldwide (NASDAQ:CHRW) Vs The Rest Of The Air Freight and Logistics Stocks : As the craze of earnings season draws to a close, here’s a look back at some of the most exciting (and some less so) results from Q1. Today, we are looking at air freight and logistics stocks, starting with C.H. Robinson Worldwide (NASDAQ:CHRW). |


In [32]:
print_markdown("# Industry-focused questions")
for question in questions_industry:
    answer, documents = enriched_rag.query(question)
    display_enriched_rag_result(question, answer, documents)

# Industry-focused questions

## Q: What are the main themes emerging in financial news about the semiconductor industry?
Emerging themes in financial news about the semiconductor industry, particularly as highlighted by the recent articles on ON Semiconductor Corp. and Analog Devices, include:

1. **Revenue Trends and International Exposure**:
   - There is a strong focus on international revenue trends for semiconductor companies, such as ON Semiconductor. Investors are keenly assessing how global market dynamics impact revenue and future forecasts. 

2. **Investor Sentiment and Stock Performance**:
   - Interest from investors is increasing for companies like ON Semiconductor, with articles noting heightened attention around stock movements. Despite some companies, including ON, facing soft earnings, investor optimism remains a theme, suggesting a belief in long-term growth potential.

3. **Earnings Season Insights**:
   - The recent earnings season has provided insights into how semiconductor companies are navigating the current business environment. Analysts are closely examining companies like Analog Devices and comparing them with peers to identify potential winners in the market.

4. **Stock Picks by Notable Investors**:
   - The mention of billionaire investors making specific stock picks, such as ON Semiconductor, indicates a trend where high-profile investors are influencing market perceptions. This adds to the momentum for certain stocks that are viewed as having significant upside potential.

5. **Market Volatility and Resilience**:
   - The semiconductor industry continues to demonstrate resilience despite challenges, as seen in the mixed earnings reports. This resilience positions semiconductor stocks as potentially lucrative investments, even in uncertain market conditions.

Overall, the semiconductor sector is characterized by a mix of cautious optimism, strategic international positioning, and increased investor interest, with particular focus on companies like ON Semiconductor and Analog Devices.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| ON | ON Semiconductor Corporation | Technology | Semiconductors | Investing in ON Semiconductor Corp. (ON)? Don't Miss Assessing Its International Revenue Trends : Explore ON Semiconductor Corp.'s (ON) international revenue trends and how these numbers impact Wall Street's forecasts and what's ahead for the stock. |
| ON | ON Semiconductor Corporation | Technology | Semiconductors | ON Semiconductor Corporation (ON) is Attracting Investor Attention: Here is What You Should Know : Recently, Zacks.com users have been paying close attention to ON Semiconductor Corp. (ON). This makes it worthwhile to examine what the stock has in store. |
| ON | ON Semiconductor Corporation | Technology | Semiconductors | Some May Be Optimistic About ON Semiconductor's (NASDAQ:ON) Earnings : Soft earnings didn't appear to concern ON Semiconductor Corporation's ( NASDAQ:ON ) shareholders over the last week... |
| ADI | Analog Devices, Inc. | Technology | Semiconductors | Spotting Winners: Vishay Intertechnology (NYSE:VSH) And Analog Semiconductors Stocks In Q1 : The end of an earnings season can be a great time to discover new stocks and assess how companies are handling the current business environment. Let’s take a look at how Vishay Intertechnology (NYSE:VSH) and the rest of the analog semiconductors stocks fared in Q1. |
| ON | ON Semiconductor Corporation | Technology | Semiconductors | ON Semiconductor (ON): Among Billionaire Glenn Russell Dubin’s Stock Picks with Huge Upside Potential : We recently published a list of Billionaire Glenn Russell Dubin’s 10 Stock Picks with Huge Upside Potential. In this article, we are going to take a look at where ON Semiconductor Corporation (NASDAQ:ON) stands against Billionaire Glenn Russell Dubin’s other stock picks with huge upside potential. Glenn Russell Dubin is one of the industry’s most […] |


## Q: What trends are being reported in the retail industry?
The recent trends in the retail industry highlight several key issues and shifts:

1. **Volatility in Consumer Spending**: Retail stocks have experienced a significant decline, with an overall drop of 13.7% over the last six months, which contrasts sharply with the S&P 500’s lesser decline of 5.5%. This indicates that consumer discretionary spending is currently facing challenges, likely linked to economic uncertainties.

2. **Adaptation to Technological Changes**: Retailers are updating their business models to better align with technological advancements that influence shopping behaviors. This includes enhancing online shopping experiences and utilizing e-commerce strategies.

3. **Impact of Trade Policies**: Retailers are adjusting their strategies in response to changing trade policies. Many have shifted their supply chains, and price increases due to tariffs have affected the pricing of goods on shelves. This suggests that external economic factors are shaping retail operations.

4. **Cyclical Nature**: The retail industry remains closely tied to economic performance and cycles. For instance, businesses in this sector, like those in lodging and consumer discretionary categories, continue to feel the pinch as demand wanes, with reports of a 12.3% decline in performance in some sectors.

Overall, retailers are navigating a challenging landscape marked by fluctuating demand, the need for technological adaptation, and the complexities arising from trade policies.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| KMX | CarMax, Inc. | Consumer Cyclical | Auto & Truck Dealerships | 3 Consumer Stocks That Concern Us : Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of consumer spending. This has stirred some uncertainty lately as retail stocks have tumbled by 13.7% over the past six months. This performance was worse than the S&P 500’s 5.5% loss. |
| BBY | Best Buy Co., Inc. | Consumer Cyclical | Specialty Retail | Retailers, Ducking Trade-War Curveballs, Stick to Their Plans : As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increases already have hit shelves. |
| HLT | Hilton Worldwide Holdings Inc. | Consumer Cyclical | Lodging | 3 Consumer Stocks Skating on Thin Ice : The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand trends are working against their favor as the industry has tumbled by 12.3%. This drop was significantly worse than the S&P 500’s 2.1% decline. |
| PKG | Packaging Corporation of America | Consumer Cyclical | Packaging & Containers | Packaging Corporation of America (NYSE:PKG) Hasn't Managed To Accelerate Its Returns : What trends should we look for it we want to identify stocks that can multiply in value over the long term? Amongst... |
| APD | Air Products and Chemicals, Inc. | Basic Materials | Specialty Chemicals | Air Products and Chemicals (NYSE:APD) Will Be Hoping To Turn Its Returns On Capital Around : To find a multi-bagger stock, what are the underlying trends we should look for in a business? Firstly, we'll want to... |


## Q: What risks or challenges are discussed in recent news about the energy industry?
Recent news highlights several significant risks and challenges facing the energy industry:

1. **Legislative Risks in Renewable Energy**: A major threat to the renewable energy sector, as exemplified by companies like NextEra Energy and Enphase Energy, is the potential repeal of critical subsidies. The U.S. House of Representatives is advancing a bill that could eliminate important financial incentives for renewable energy projects, which may render many of these initiatives uneconomical, negatively impacting investments and growth in the sector.

2. **Volatility in Oil Prices**: For companies in the oilfield service sector such as Halliburton and Baker Hughes, the slide in oil prices presents a substantial challenge. As oil prices fluctuate downward, many companies face reduced revenue, impacting drilling budgets and operational viability.

3. **Tariffs and Trade Concerns**: Rising tariffs create an uncertain environment for energy companies reliant on international trade, further exacerbating the situation for firms within the oil and gas equipment/services sectors. These tariffs can lead to increased costs and reduced competitiveness in a volatile market.

4. **Budget Constraints**: The reduction in drilling budgets, driven by the aforementioned factors, could lead to a slowdown in exploration and production activities, further challenging the financial health of energy companies.

5. **Demand Uncertainty**: While there are emerging opportunities in liquefied natural gas (LNG) and artificial intelligence (AI) applications, it remains to be seen whether these can sufficiently compensate for the declining oil prices and stricter regulations.

In summary, the energy industry is grappling with legislative, economic, and market pressures that could significantly alter its landscape in the near future.

### The following news articles were used for generating this answer:
| Ticker | Company | Sector | Industry | Article |
|--------|--------------|--------|----------|-----|
| NEE | NextEra Energy, Inc. | Utilities | Utilities - Regulated Electric | Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law. |
| ENPH | Enphase Energy, Inc. | Technology | Solar | Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law. |
| HAL | Halliburton Company | Energy | Oil & Gas Equipment & Services | Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support? |
| BKR | Baker Hughes Company | Energy | Oil & Gas Equipment & Services | Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support? |
| FCX | Freeport-McMoRan Inc. | Basic Materials | Copper | 3 American Companies Investors Need to Know Amid Trump's Tariff Wars : Copper is a critical metal for the U.S. industrial economy.  This American appliance maker expects the Trump administration to close loopholes that will improve its competitive positioning.  It's difficult to predict precisely what the tariff landscape will look like when the dust settles on the trade conflict, but we can say some things with a high degree of certainty. |


## Analysis & Questions - Section 2

### Instructions: Evaluate Answers With and Without Metadata

For each question, compare the two answers provided:
- One generated **without** metadata
- One generated **with** metadata

---

### Steps:

1. Use the following evaluation criteria:
   - Clarity
   - Detail & Depth
   - Use of Context
   - Accuracy & Grounding
   - Relevance
   - Narrrative Flow

2. For each criterion, write brief notes comparing how the answer **without metadata** performs versus the answer **with metadata**.

3. Summarize your evaluation in a markdown table with the following columns:

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [Your brief note here]     | [Your brief note here]   |
| Detail & Depth         | [Your brief note here]     | [Your brief note here]   |
| Use of Context        | [Your brief note here]     | [Your brief note here]   |
| Accuracy & Grounding       | [Your brief note here]     | [Your brief note here]   |
| Relevance      | [Your brief note here]     | [Your brief note here]   |
| Narrative Flow      | [Your brief note here]     | [Your brief note here]   |

---

**Note:** Keep comments short and clear for easy comparison.



YOUR WRITTEN RESPONSE HERE